# Transformation des données du climat

## Données du climats

In [2]:
import pandas as pd 
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns 

pd.options.display.max_columns = None
pd.options.display.max_rows = None
# pd.options.display.float_format = '{:.2f}'.format 

# Lecture des données

In [3]:
weather = pd.read_csv ('Data/donnees-synop-essentielles-omm.csv', sep = ';')

In [4]:
print(weather.columns.values)

['ID OMM station' 'Date' 'Pression au niveau mer'
 'Variation de pression en 3 heures' 'Type de tendance barométrique'
 'Direction du vent moyen 10 mn' 'Vitesse du vent moyen 10 mn'
 'Température' 'Point de rosée' 'Humidité' 'Visibilité horizontale'
 'Temps présent' 'Temps passé 1' 'Temps passé 2' 'Nebulosité totale'
 "Nébulosité  des nuages de l' étage inférieur"
 "Hauteur de la base des nuages de l'étage inférieur"
 "Type des nuages de l'étage inférieur" "Type des nuages de l'étage moyen"
 "Type des nuages de l'étage supérieur" 'Pression station'
 'Niveau barométrique' 'Géopotentiel' 'Variation de pression en 24 heures'
 'Température minimale sur 12 heures' 'Température minimale sur 24 heures'
 'Température maximale sur 12 heures' 'Température maximale sur 24 heures'
 'Température minimale du sol sur 12 heures'
 'Méthode de mesure Température du thermomètre mouillé'
 'Température du thermomètre mouillé'
 'Rafale sur les 10 dernières minutes' 'Rafales sur une période'
 'Periode de mes

Le nombre de colonnes est assez important. Nous allons en selectionner qu'un sous ensemble ayant un lien à priori avec la consommation d'energie

# Sélection du périmètre 
1) on ne conserve qu'un périmètre temporel précis. Pour l'instant 2015 - 2019

2) On supprime certaines régions qui ne font pas partie du périmètre de données du fichier des données electricité (guadeloupe, martinique , ...)

3) il y a des données qui sont référencées dans des régions qui n'existent pas dans le fichier des départements. On les supprime aussi (984 , 977 , 975)


In [5]:
weather = weather[(weather['Date'] >= '2015') & (weather['Date'] <= '2020') ]  

In [6]:
weather = weather[~( (weather['region (code)'] == 1.00) | (weather['region (code)'] == 2.00) | (weather['region (code)'] == 3.00)
                 | (weather['region (code)'] == 4.00) | (weather['region (code)'] == 5.00) | (weather['region (code)'] == 6.00) )] 


In [7]:
pd.crosstab(weather['region (code)'],weather['region (code)'].count())

col_0,631952
region (code),
11.0,14583
24.0,29155
27.0,14479
28.0,57699
32.0,29138
44.0,72736
52.0,14583
53.0,58150
75.0,72335


In [8]:
weather = weather[~( (weather['region (code)'] == 975.00 ) | (weather['region (code)'] ==977.00) | (weather['region (code)'] == 984.00))] 

# selection de colonnes et transformation

1) Selection des colonnes

In [9]:
weather = weather[['Date','ID OMM station','Température (°C)','Température minimale sur 12 heures (°C)','Température minimale sur 24 heures (°C)'
                       ,'Température maximale sur 12 heures (°C)','Température maximale sur 24 heures (°C)'
                       ,'Humidité','Nebulosité totale','Rafales sur une période','Periode de mesure de la rafale'
                       ,'Vitesse du vent moyen 10 mn','Précipitations dans la dernière heure'
                       ,'Précipitations dans les 3 dernières heures'
                       , 'Précipitations dans les 6 dernières heures','Précipitations dans les 12 dernières heures' 
                       ,'Précipitations dans les 24 dernières heures' 
                       ,'region (code)']]

2) Transformation des colonnes de date

In [10]:
weather['aaaammjj'] = pd.to_datetime(weather['Date'].apply(lambda get_year: get_year[0:10] ) )

weather['annee'] =  pd.to_datetime(weather['aaaammjj']).dt.year
weather['mois'] = pd.to_datetime(weather['aaaammjj']).dt.month
weather['joursem'] = pd.to_datetime(weather['aaaammjj']).dt.dayofweek
weather['jour'] = pd.to_datetime(weather['aaaammjj']).dt.day

weather['hhmmss'] =  weather['Date'].apply(lambda get_year: get_year[11:19] )



3) Analyse de la variable 'Periode de mesure de la rafale' 
Il y a 98% des données qui ont la meme valeur. Cette colonne ne servira donc à rien. On la supprime

In [11]:
weather['Periode de mesure de la rafale'].value_counts(normalize = True)

-10.0    0.988715
-60.0    0.005827
-30.0    0.005458
Name: Periode de mesure de la rafale, dtype: float64

4. Suppression des colonnes inutiles 

In [12]:
weather = weather[['Date','ID OMM station','Température (°C)','Température minimale sur 12 heures (°C)','Température minimale sur 24 heures (°C)'
                       ,'Température maximale sur 12 heures (°C)','Température maximale sur 24 heures (°C)'
                       ,'Humidité','Nebulosité totale','Rafales sur une période','Vitesse du vent moyen 10 mn','Précipitations dans la dernière heure'
                       ,'Précipitations dans les 3 dernières heures'
                       , 'Précipitations dans les 6 dernières heures','Précipitations dans les 12 dernières heures' 
                       ,'Précipitations dans les 24 dernières heures' 
                       ,'region (code)','aaaammjj','annee','mois','joursem','jour','hhmmss']]

5. Les précipitations < 0 sont remises à zéro 

In [13]:
weather ['Précipitations dans la dernière heure'] [(weather['Précipitations dans la dernière heure'] < 0) ] = 0 
weather ['Précipitations dans les 3 dernières heures'] [(weather['Précipitations dans les 3 dernières heures'] < 0) ] = 0 
weather ['Précipitations dans les 6 dernières heures'] [(weather['Précipitations dans les 6 dernières heures'] < 0) ] = 0 
weather ['Précipitations dans les 12 dernières heures'] [(weather['Précipitations dans les 12 dernières heures'] < 0) ] = 0 
weather ['Précipitations dans les 24 dernières heures'] [(weather['Précipitations dans les 24 dernières heures'] < 0) ] = 0

# Traitement des id_station

Il y a un certain nombre d'enregistrements pour lesquels il n'y a pas de région. 
On va rechercher la région via le fichier des station méteo et celui des communes. 

Dans le fichier des stations, la donnée de la ville est souvent constituée de la ville - Quartier (exemple LILLE-LESQUIN)     

On fera donc 2 jointures distinctes entre station et ville : Une ou la Ville (station) = ville (commune)   
                                                             l'autre ou ville (station - le quartier) = ville de commune.   
                                                              On prendra le résultat de la jointure qui n'est pas nuelle  
On ne peut pas faire l'opération en une seule fois car il existe des villes avec un "-" dans le nom.

In [14]:
station = pd.read_csv ('Data/stations_meteo.csv', sep = ';')
station.head()

,ID,Nom,Latitude,Longitude,Altitude
0,7005,ABBEVILLE,50.136000,1.834000,69
1,7015,LILLE-LESQUIN,50.570000,3.097500,47
2,7020,PTE DE LA HAGUE,49.725167,-1.939833,6
3,7027,CAEN-CARPIQUET,49.180000,-0.456167,67
4,7037,ROUEN-BOOS,49.383000,1.181667,151


In [15]:
# On calcul le nom de la ville sans les "-" 
station['nom_alt'] =station['Nom'].apply(lambda get_ville : get_ville if get_ville.find("-") == -1 else get_ville[0:get_ville.find("-")] )

In [16]:
station.head()

,ID,Nom,Latitude,Longitude,Altitude,nom_alt
0,7005,ABBEVILLE,50.136000,1.834000,69,ABBEVILLE
1,7015,LILLE-LESQUIN,50.570000,3.097500,47,LILLE
2,7020,PTE DE LA HAGUE,49.725167,-1.939833,6,PTE DE LA HAGUE
3,7027,CAEN-CARPIQUET,49.180000,-0.456167,67,CAEN
4,7037,ROUEN-BOOS,49.383000,1.181667,151,ROUEN


In [17]:
commune =   pd.read_csv ('Data/communes2020.csv', sep = ',')

#### Premiere jointure avec le nom sans modification

In [18]:
station = pd.merge (station,commune[(commune['typecom'] == 'COM')], left_on = 'Nom', right_on = 'ncc' , how='left')

In [19]:
station = station [['ID','Nom','reg','ncc','nom_alt']]

station.head()

,ID,Nom,reg,ncc,nom_alt
0,7005,ABBEVILLE,32.0,ABBEVILLE,ABBEVILLE
1,7015,LILLE-LESQUIN,NaN,NaN,LILLE
2,7020,PTE DE LA HAGUE,NaN,NaN,PTE DE LA HAGUE
3,7027,CAEN-CARPIQUET,NaN,NaN,CAEN
4,7037,ROUEN-BOOS,NaN,NaN,ROUEN


On supprime les doublons

In [20]:
station = station.drop_duplicates(subset='Nom', keep='first')

On sauvegarde les données pour lesquelles on a bien trouver la ville 

In [21]:
station_v1 =  (station[~(station['reg'].isna())] )

In [22]:
station_v1.head()

,ID,Nom,reg,ncc,nom_alt
0,7005,ABBEVILLE,32.0,ABBEVILLE,ABBEVILLE
9,7139,ALENCON,28.0,ALENCON,ALENCON
10,7149,ORLY,11.0,ORLY,ORLY
16,7240,TOURS,24.0,TOURS,TOURS
17,7255,BOURGES,24.0,BOURGES,BOURGES


On prend les NAN restant afin d'essayer la seconde jointure avec le nom modifié

In [23]:
station =  (station[(station['reg'].isna())] )
station = station [['ID','Nom','nom_alt']]

In [24]:
station = pd.merge (station,commune[(commune['typecom'] == 'COM')], left_on = 'nom_alt', right_on = 'ncc' , how='left')

In [25]:
station = station [['ID','Nom','reg','ncc','nom_alt']]
station.head()

,ID,Nom,reg,ncc,nom_alt
0,7015,LILLE-LESQUIN,32.0,LILLE,LILLE
1,7020,PTE DE LA HAGUE,NaN,NaN,PTE DE LA HAGUE
2,7027,CAEN-CARPIQUET,28.0,CAEN,CAEN
3,7037,ROUEN-BOOS,28.0,ROUEN,ROUEN
4,7072,REIMS-PRUNAY,44.0,REIMS,REIMS


On supprime tous les doublons 

In [26]:
station = station.drop_duplicates(subset='Nom', keep=False)
station =  (station[~(station['reg'].isna())] )

On reconstruit station avec l'ensemble des stations pour lesquelles on a trouver une ville

In [27]:
station = pd.concat([ station,station_v1] , axis = 0 )

In [28]:
del station_v1

On réalise la jointure entre les stations et les données météo

In [29]:
weather2 = pd.merge (weather, station, left_on = ['ID OMM station'], right_on = ['ID'], how= 'left')

Mauvaise surprise il y a des stations qui n'ont toujours pas de région 

In [30]:
weather2['ID OMM station'][weather2['region (code)'].isna()].value_counts()

89642    12714
61996    12066
61968    10434
61997    10172
71805     9688
61998     8884
61976     7305
61972     5304
78894     1560
61970      615
Name: ID OMM station, dtype: int64

Via Google maps on regarde ou sont localisées ces stations 

station
  id 
89642  ===> Base antartique DUMONT D'URVILLE  
61996  ===> NOUVELLE AMSTERDAM (ile au milieu de nulle part)   
61968  ===> Les iles Grandes glorieuses ===> Hors sujet   
61997  ===> Crozet - Ile de la possession   
71805  ===> ST-PIERRE (du coté des états unis)  
61998  ===> Ile Kergelen   
61976  ===> Ile de TROMELIN ... C'est encore plus petit !!   
61972  ===> Ile Europa à l'est de MAdagascar   
78894  ===> ST-BARTHELEMY METEO  
61970  ====> Un peu plus au nord de Madagascar   
 
Donc on peut simplement  supprimer les enregistrements pour lesquels les régions sont vides

In [31]:
print ( 'weather.shape : ' ,weather.shape)
weather = weather.dropna ( subset = ['region (code)'])
print ('weather.shape : ' ,weather.shape)
del weather2

weather.shape :  (688136, 23)
weather.shape :  (609394, 23)


# Traitement des NAN 
* Précipitations on ne peut pas savoir ... On met à zéro 
* Humidité moyenne par rapport à la ligne d'avant et d'après 
* Nebulosité totale :  moyenne par rapport à la ligne d'avant et d'après 
             A voir si pour les NAN : il ne faudrait pas commencer par mettre les tranches de 1H00 et 4:00 à 100% avant de calculer la moyenne  
* température : Moyenne 
       

In [32]:
weather[['Précipitations dans les 3 dernières heures','Précipitations dans les 6 dernières heures','Précipitations dans les 12 dernières heures',
         'Précipitations dans les 24 dernières heures','Précipitations dans la dernière heure']] = weather[['Précipitations dans les 3 dernières heures','Précipitations dans les 6 dernières heures',
                                                                    'Précipitations dans les 12 dernières heures','Précipitations dans les 24 dernières heures','Précipitations dans la dernière heure']].fillna(0) 


Création d'une seconde colonne de nebulosité pour tester l'effet de la mise  à 100 des données de pleine nuit


In [33]:
weather['Nebulosite totale 2'] = weather['Nebulosité totale']
weather['Nebulosite totale 2'][(weather['Nebulosite totale 2'].isna())&(weather['hhmmss'] == '01:00:00')].isna().sum()
weather['Nebulosite totale 2'][(weather['Nebulosite totale 2'].isna())&(weather['hhmmss'] == '04:00:00')].isna().sum()

13450

In [34]:

weather['Nebulosite totale 2'][(weather['Nebulosite totale 2'].isna())&(weather['hhmmss'] == '01:00:00')] = 100
weather['Nebulosite totale 2'][(weather['Nebulosite totale 2'].isna())&(weather['hhmmss'] == '04:00:00')]  = 100

C:\Users\bente\AppData\Local\Temp\ipykernel_4844\1192439244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['Nebulosite totale 2'][(weather['Nebulosite totale 2'].isna())&(weather['hhmmss'] == '01:00:00')] = 100
C:\Users\bente\AppData\Local\Temp\ipykernel_4844\1192439244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['Nebulosite totale 2'][(weather['Nebulosite totale 2'].isna())&(weather['hhmmss'] == '04:00:00')]  = 100


In [35]:
weather.isna().sum()/weather.shape[0]

Date                                           0.000000
ID OMM station                                 0.000000
Température (°C)                               0.001774
Température minimale sur 12 heures (°C)        0.757689
Température minimale sur 24 heures (°C)        1.000000
Température maximale sur 12 heures (°C)        0.757686
Température maximale sur 24 heures (°C)        1.000000
Humidité                                       0.002273
Nebulosité totale                              0.457213
Rafales sur une période                        0.030745
Vitesse du vent moyen 10 mn                    0.002982
Précipitations dans la dernière heure          0.000000
Précipitations dans les 3 dernières heures     0.000000
Précipitations dans les 6 dernières heures     0.000000
Précipitations dans les 12 dernières heures    0.000000
Précipitations dans les 24 dernières heures    0.000000
region (code)                                  0.000000
aaaammjj                                       0

Concernant les données de températures minimales, il y a beaucoup trop de NA 
On supprime les colonnes

In [36]:
weather = weather.drop (['Température minimale sur 24 heures (°C)','Température minimale sur 12 heures (°C)','Température maximale sur 24 heures (°C)','Température maximale sur 12 heures (°C)'], axis = 1)

Pour calculer les moyennes on commence par trier le df par région et par date et Heure

In [37]:
weather = weather.sort_values(by=['ID OMM station','Date']).reset_index()
weather = weather.drop ('index', axis = 1)

Par itération pour chaque colonne on va chercher pour chaque ligne la valeur d'avant et celle d'après et on en fait une moyenne 

remarque : Cette étape est un peu longue (environ 10 minutes sur mon pc)  


In [38]:
# Paramètrage de la boucle 

column_list = ['Température (°C)','Humidité','Nebulosité totale','Nebulosite totale 2','Rafales sur une période','Vitesse du vent moyen 10 mn']

cpt = 0

print ('nombre totales de NAN à traiter : ',weather[column_list].isna().sum().sum() )  

nombre totales de NAN à traiter :  552186


In [39]:
start = time.time()
print (' début : ' ,start)
    
for ma_col in column_list:
    cpt +=1
    print ('colonne traitée :',ma_col)
        
    for mon_nan in weather[(weather[ma_col].isna() == True)].index :
        cpt +=1
        if mon_nan < (weather.shape[0]-1) and mon_nan > 0 :
            weather[ma_col].loc[mon_nan] = weather[ma_col].loc[mon_nan -1 : mon_nan +1].mean()
                

        if cpt == 100000: 
            print (weather.shape)
            print (f'{cpt} lignes traitées - Temps d execution {time.time() - start}') 
            cpt = 0   

            print ('nombre de NAN restant  : ',weather[column_list].isna().sum().sum() )  
  
print(' Fin : ' ,time.time() - start)

 début :  1657887069.500093
colonne traitée : Température (°C)


C:\Users\bente\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


colonne traitée : Humidité
colonne traitée : Nebulosité totale
(609394, 20)
100000 lignes traitées - Temps d execution 74.31474542617798
nombre de NAN restant  :  452190
(609394, 20)
100000 lignes traitées - Temps d execution 149.38433051109314
nombre de NAN restant  :  352190
colonne traitée : Nebulosite totale 2
(609394, 20)
100000 lignes traitées - Temps d execution 225.10238933563232
nombre de NAN restant  :  252191
(609394, 20)
100000 lignes traitées - Temps d execution 301.610155582428
nombre de NAN restant  :  152191
(609394, 20)
100000 lignes traitées - Temps d execution 379.8134808540344
nombre de NAN restant  :  52191
colonne traitée : Rafales sur une période
colonne traitée : Vitesse du vent moyen 10 mn
 Fin :  419.20062232017517


### controles post traitement   

il reste une seule ligne qui concerne la nébulosité : Il s'agit du premier enregistrement du fichier à 1 heure du matin On met la ligne à 100 

In [40]:
print ('nombre totals de NAN post traitement  : ',weather.isna().sum() )  

nombre totals de NAN post traitement  :  Date                                           0
ID OMM station                                 0
Température (°C)                               0
Humidité                                       0
Nebulosité totale                              1
Rafales sur une période                        0
Vitesse du vent moyen 10 mn                    0
Précipitations dans la dernière heure          0
Précipitations dans les 3 dernières heures     0
Précipitations dans les 6 dernières heures     0
Précipitations dans les 12 dernières heures    0
Précipitations dans les 24 dernières heures    0
region (code)                                  0
aaaammjj                                       0
annee                                          0
mois                                           0
joursem                                        0
jour                                           0
hhmmss                                         0
Nebulosite totale 2         

In [41]:
weather[(weather['Nebulosité totale'].isna())].head()

,Date,ID OMM station,Température (°C),Humidité,Nebulosité totale,Rafales sur une période,Vitesse du vent moyen 10 mn,Précipitations dans la dernière heure,Précipitations dans les 3 dernières heures,Précipitations dans les 6 dernières heures,Précipitations dans les 12 dernières heures,Précipitations dans les 24 dernières heures,region (code),aaaammjj,annee,mois,joursem,jour,hhmmss,Nebulosite totale 2
0,2015-01-01T01:00:00+01:00,7005,-0.6,89.0,NaN,3.9,2.8,0.0,0.0,0.0,0.0,0.0,32.0,2015-01-01,2015,1,3,1,01:00:00,100.0


In [42]:
weather['Nebulosité totale'].fillna(100, inplace = True)

weather[(weather['Nebulosité totale'].isna())].head()

,Date,ID OMM station,Température (°C),Humidité,Nebulosité totale,Rafales sur une période,Vitesse du vent moyen 10 mn,Précipitations dans la dernière heure,Précipitations dans les 3 dernières heures,Précipitations dans les 6 dernières heures,Précipitations dans les 12 dernières heures,Précipitations dans les 24 dernières heures,region (code),aaaammjj,annee,mois,joursem,jour,hhmmss,Nebulosite totale 2


## Changement des noms des colonnes 
weather = weather.rename ({'Température (°C)' : 'Temp', 'Humidité' : 'humidite', 'Nebulosité totale' : 'Nebulosité'})

In [43]:
weather = weather.rename ({'Température (°C)' : 'temp', 'Humidité' : 'humidite', 'Nebulosité totale' : 'nebulosite',
                           'Rafales sur une période' : 'rafale_periode' , 'Nebulosite totale 2' :'nebulosite2',
                         'Periode de mesure de la rafale' : 'periode_mesure_rafale' ,'Vitesse du vent moyen 10 mn' : 'vitesse_vent',
                           'Précipitations dans la dernière heure' : 'pluie_1h',
                         'Précipitations dans les 3 dernières heures' : 'pluie_3h','Précipitations dans les 6 dernières heures' : 'pluie_6h',
                         'Précipitations dans les 12 dernières heures' :  'pluie_12h', 'Précipitations dans les 24 dernières heures' :'pluie_24h',
                         'region (code)' : 'region'}, axis = 1)
weather.head()

,Date,ID OMM station,temp,humidite,nebulosite,rafale_periode,vitesse_vent,pluie_1h,pluie_3h,pluie_6h,pluie_12h,pluie_24h,region,aaaammjj,annee,mois,joursem,jour,hhmmss,nebulosite2
0,2015-01-01T01:00:00+01:00,7005,-0.6,89.0,100.0,3.9,2.8,0.0,0.0,0.0,0.0,0.0,32.0,2015-01-01,2015,1,3,1,01:00:00,100.0
1,2015-01-01T04:00:00+01:00,7005,-0.8,88.0,0.0,5.1,3.2,0.0,0.0,0.0,0.0,0.0,32.0,2015-01-01,2015,1,3,1,04:00:00,100.0
2,2015-01-01T07:00:00+01:00,7005,-0.3,84.0,0.0,5.0,3.4,0.0,0.0,0.0,0.0,0.0,32.0,2015-01-01,2015,1,3,1,07:00:00,0.0
3,2015-01-01T10:00:00+01:00,7005,1.1,78.0,25.0,5.7,3.2,0.0,0.0,0.0,0.0,0.0,32.0,2015-01-01,2015,1,3,1,10:00:00,25.0
4,2015-01-01T13:00:00+01:00,7005,5.3,60.0,90.0,10.2,5.4,0.0,0.0,0.0,0.0,0.0,32.0,2015-01-01,2015,1,3,1,13:00:00,90.0


### Aggrégation des données 

In [44]:
weather_agg = weather.groupby (['region','aaaammjj','annee','mois','joursem','jour'], as_index=False ).agg({'temp':'mean','humidite': 'mean', 'nebulosite':'mean',
                                                                               'nebulosite2':'mean','rafale_periode':'mean','vitesse_vent':'mean',
                                                                               'pluie_1h': 'sum',  'pluie_3h': 'sum','pluie_6h': 'sum','pluie_12h': 'sum',
                                                                                'pluie_24h': 'sum'})


print (weather_agg.shape)

(23737, 17)


### Modification des types de données

In [45]:
weather_agg = weather_agg.astype({'region':'int'})

### quelques vérifications 

In [46]:
# quelques vérifications 
pd.crosstab(weather_agg['annee'], weather_agg['annee'].count())

pd.crosstab(weather_agg['region'], weather_agg['annee'].count())

# pd.crosstab(weather_agg['pluie_1h'], weather_agg['pluie_1h'].sum())

col_0,23737
region,
11,1826
24,1826
27,1825
28,1826
32,1826
44,1826
52,1826
53,1826
75,1826


In [47]:
weather_agg.head()

,region,aaaammjj,annee,mois,joursem,jour,temp,humidite,nebulosite,nebulosite2,rafale_periode,vitesse_vent,pluie_1h,pluie_3h,pluie_6h,pluie_12h,pluie_24h
0,11,2015-01-01,2015,1,3,1,0.4000,84.250,30.6250,30.625,3.5875,1.8500,0.0,0.0,0.0,0.0,0.0
1,11,2015-01-02,2015,1,4,2,4.0625,88.375,73.7500,73.750,6.1250,3.2625,0.0,0.2,0.4,0.8,1.2
2,11,2015-01-03,2015,1,5,3,6.9250,93.125,92.0625,97.625,9.2750,4.5625,0.8,3.0,6.0,10.4,12.2
3,11,2015-01-04,2015,1,6,4,4.3250,83.750,72.5000,72.500,4.0500,2.2625,0.2,1.2,2.4,6.4,21.8
4,11,2015-01-05,2015,1,0,5,-0.3875,95.250,77.1250,77.125,2.3875,1.1375,0.0,0.0,0.0,0.0,0.0


### Sortie des données sous forme de fichier 

In [48]:
weather_agg.to_csv ('Data/weather_out.csv',index=False)